In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_key = os.environ['OPENAI_API_KEY']
serp_key = os.environ["SERPAPI_API_KEY"]

In [2]:
from langchain.chat_models import ChatOpenAI 
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
llm= ChatOpenAI() #select the model 

from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool

serp_search = SerpAPIWrapper()


#search_tool 
serp_search_tool = Tool(
    name="Search",
    func=serp_search.run,
    description="Useful for when you need to answer questions about current events. You should ask targeted questions"
)

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

raw_document = PyPDFLoader('tunisia_travler_guide.pdf').load()

#texter splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap=200
)

document_splitted = text_splitter.split_documents(raw_document)
db = FAISS.from_documents(document_splitted,OpenAIEmbeddings())

from langchain.agents.agent_toolkits import create_retriever_tool

#tunisia_retrival_tool
tunisia_retrival_tool = create_retriever_tool(
 db.as_retriever(),
 "tunisia_travel",
 "Searches and returns documents regarding Tunisia."
)


tools = [serp_search_tool,tunisia_retrival_tool]


tunisia_guide = create_conversational_retrieval_agent(
  llm,tools=tools, memory_key='chat_history'
) #create our agent the memory argument is what will make the agent keep track 
# of the chat history

C:\Users\hajme\AppData\Local\Temp\ipykernel_24220\2581345455.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm= ChatOpenAI() #select the model
C:\Users\hajme\AppData\Local\Temp\ipykernel_24220\2581345455.py:32: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(document_splitted,OpenAIEmbeddings())


In [9]:
response =tunisia_guide({"input": "I want to spend 2 days in Tunis and Sousse"})
print(response['output'])

Spending 2 days in both Tunis and Sousse can give you a taste of Tunisia's diverse offerings, from historical sites to beach relaxation. Here is a suggested itinerary for your 2-day trip to Tunis and Sousse:

Day 1:
- Morning: Start your day in Tunis by exploring the historic Medina, visiting landmarks like the Zitouna Mosque and shopping for souvenirs in the bustling souks.
- Lunch: Enjoy a traditional Tunisian meal at a local restaurant in the Medina.
- Afternoon: Visit the Bardo Museum to admire its collection of Roman mosaics and artifacts.
- Evening: Head to Sidi Bou Said, a picturesque village overlooking the Mediterranean Sea, for a leisurely stroll and a cup of mint tea.

Day 2:
- Morning: Travel to Sousse, a coastal city known for its beaches and historical sites. Visit the Medina of Sousse, a UNESCO World Heritage site, and explore its narrow streets and traditional architecture.
- Lunch: Enjoy fresh seafood at a restaurant overlooking the sea in Sousse.
- Afternoon: Visit th

In [5]:
print(response['output'])

The three top most visited cities in Tunisia are:

1. Tunis: Known for its Avenue Habib Bourguiba, one of the longest streets in Tunis, lined with cafes, stores, modern hotels, and colonial-style buildings. It is popularly known as the Champs Elysees of Tunisia.
2. Kairouan: A mid-sized city with attractions like the Medina (old market), The Great Mosque (one of the oldest in the world), and Sidi Saheb Zaouia ("Mosque of the Barber") which is home to the tomb of a companion of the Prophet.
3. Tunis (again): Mentioned for the Zitouna Mosque, a fascinating fusion of buildings from different periods, with columns and capitals from ancient Carthage, an Ottoman minaret, and Zirid domes. Non-Muslims are only permitted to access the central courtyard of the mosque.
